# Pytorch Tutorial

Pytorch - фреймворк на питоне для машинного / глубокого обучения

- Ускоренные на GPU операции
- Автоматическое дифференцирование
- Модули для нейронных сетей

Этот урок научит вас основам работы с тензорами и сетями в Pytorch.

![](karpathy.png)

Установка https://pytorch.org/get-started

In [ ]:
import torch
import numpy as np
import torch.nn as nn

# если гпу у вас несколько укажите номер гпу, которую хотите использовать
# номер свободной можно посмотреть с помощью nvidia-smi в терминале
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

## Tensors (review)

Тензоры являются фундаментальным объектом для массивов данных. Наиболее распространенные типы, которые вы будете использовать, это `IntTensor` и `FloatTensor`.

In [ ]:
# Создаем пустой тензор
x = torch.FloatTensor(2,3)
print(x)
# Инициализируем нулями
x.zero_()
print(x)

In [ ]:
# Создаем numpy array (seed для воспроизведения результатов)
np.random.seed(123)
np_array = np.random.random((2,3))
print(torch.FloatTensor(np_array))
print(torch.from_numpy(np_array))

In [ ]:
# Создаем тензор с случайными данными (seed для воспроизведения результатов)
torch.manual_seed(123)
x=torch.randn(2,3)
print(x)
# Экспортируем numpy array
x_np = x.numpy()
print(x_np)

In [ ]:
# Особые тензоры (смотрим документацию)
print(torch.eye(3))
print(torch.ones(2,3))
print(torch.zeros(2,3))
print(torch.arange(0,3))

Все тензоры имеют размер `size` и тип `type`

In [ ]:
x=torch.FloatTensor(3,4)
print(x.size())
print(x.type())

## Математика, Линейная алгебра, и индексирование (обзор)

Математика Pytorch и линейная алгебра в Pytorch похожа на NumPy. Операторы переопределяются, поэтому вы можете использовать стандартные математические операторы (`+`, `-` и т.д.) и ожидать в результате тензор. Смотрим документацию Pytorch для полного списка доступных функций.

In [ ]:
x = torch.arange(0,5).float()
print(torch.sum(x))
print(torch.sum(torch.exp(x)))
print(torch.mean(x))

Индексирование в Pytorch похоже на индексирование в numpy. Смотрим документацию Pytorch для деталей.

In [ ]:
x = torch.rand(3,2)
print(x)
print(x[1,:])

## CPU и GPU

Тензор можно копировать между CPU и GPU. Важно, чтобы все, что участвует в расчете, было на одном устройстве.

Эта часть кода может не сработать, если у вас нет доступного GPU.

In [ ]:
# create a tensor
x = torch.rand(3,2)
print(x)
# copy to GPU
y = x.cuda()
print(y)
# copy back to CPU
z = y.cpu()
print(z)
# get CPU tensor as numpy array
print(z.numpy())
# cannot get GPU tensor as numpy array directly
try:
    y.numpy()
except RuntimeError as e:
    print(e)

Операции между тензорами на GPU и CPU не будут выполнены. Операции требуют, чтобы все аргументы были на одном устройстве.

In [ ]:
x = torch.rand(3,5)  # CPU tensor
y = torch.rand(5,4).cuda()  # GPU tensor
try:
    torch.mm(x,y)  # Operation between CPU and GPU fails
except TypeError as e:
    print(e)

Типичный код должен включать в себя операторы `if` или использовать вспомогательные функции, чтобы он мог работать с GPU или без него.

In [ ]:
# Put tensor on CUDA if available
x = torch.rand(3,2)
if torch.cuda.is_available():
    x = x.cuda()

# Do some calculations
y = x ** 2 

# Copy to CPU if on GPU
if y.is_cuda:
    y = y.cpu()

Удобным методом является `new`, который создает новый тензор на том же устройстве, что и другой тензор. Его следует использовать для создания тензоров, когда это возможно.

In [ ]:
x1 = torch.rand(3,2)
x2 = x1.new(1,2)  # create cpu tensor
print(x2)
x1 = torch.rand(3,2).cuda()
x2 = x1.new(1,2)  # create cuda tensor
print(x2)

Расчеты, выполняемые на GPU, могут быть во много раз быстрее, чем просто в numpy. Однако numpy по-прежнему оптимизирован для процессора и во много раз быстрее, чем циклы python `for`. Numpy вычисления могут быть быстрее, чем вычисления GPU для небольших массивов из-за стоимости взаимодействия с GPU.

In [ ]:
from timeit import timeit
# Create random data
x = torch.rand(1000,64)
y = torch.rand(64,32)
number = 10000  # number of iterations

def square():
    z=torch.mm(x, y) # dot product (mm=matrix multiplication)

# Time CPU
print('CPU: {}ms'.format(timeit(square, number=number)*1000))
# Time GPU
x, y = x.cuda(), y.cuda()
print('GPU: {}ms'.format(timeit(square, number=number)*1000))

## Дифференцирование

Тензоры обеспечивают автоматическое дифференцирование

Что нужно знать:

- Тензоры, по которым вы дифференцируетесь, должны иметь `require_grad = True`
- Вызвать `.backward ()` для скалярных переменных, которые вы дифференцируете
- Чтобы дифференцировать вектор, сначала суммируйте его

In [ ]:
# Create differentiable tensor
x = torch.tensor(torch.arange(0,4).float(), requires_grad=True)
# Calculate y=sum(x**2)
y = x**2
# Calculate gradient (dy/dx=2x)
y.sum().backward()
# Print values
print(x)
print(y)
print(x.grad)

Дифференцирование накапливает градиенты. Иногда это то, что вы хотите, а иногда нет. **Обязательно обнуляйте градиенты между батчами при выполнении градиентного спуска, иначе вы получите странные результаты!**

In [ ]:
# Create a variable
x=torch.tensor(torch.arange(0,4).float(), requires_grad=True)
# Differentiate
torch.sum(x**2).backward()
print(x.grad)
# Differentiate again (accumulates gradient)
torch.sum(x**2).backward()
print(x.grad)
# Zero gradient before differentiating
x.grad.data.zero_()
torch.sum(x**2).backward()
print(x.grad)

Обратите внимание, что Тензор с градиентом не может быть экспортирован напрямую в numpy:

In [ ]:
x=torch.tensor(torch.arange(0,4).float(), requires_grad=True)
x.numpy() # raises an exception

Причина в том, что pytorch запоминает граф всех вычислений для выполнения дифференцирования. Для интеграции в этот граф необработанные данные внутренне обертываются в класс Tensor (как, например, переменная). Вы можете отсоединить тензор от графа, используя метод **.detach()**, который возвращает тензор с теми же данными, но для параметра require_grad установлено значение False.

In [ ]:
x=torch.tensor(torch.arange(0,4).float(), requires_grad=True)
y=x**2
z=y**2
z.detach().numpy()

Другая причина использовать этот метод заключается в том, что для обновления графа может потребоваться много памяти. Если вы находитесь в ситуации, где у вас есть тензор который вам не нужно дифференцировать, подумайте об отделении его от графа.

## Neural Network Modules

Pytorch предоставляет основу для разработки модулей нейронной сети. Они заботятся о многих вещах, главной из которых является упаковка и отслеживание списка параметров для вас.
У вас есть несколько способов построения и использования сети, предлагая различные компромиссы между свободой и простотой.

torch.nn предоставляет базовые однослойные сети, такие как линейные (перцептрон) и слои активации.

In [ ]:
x = torch.arange(0,32).float()
net = torch.nn.Linear(32,10)
y = net(x)
print(y)

Все объекты nn.Module могут использоваться в качестве компонентов больших сетей! Вот как можно построить свою собственную сеть. Самый простой способ - использовать класс nn.Sequential.

Вы также можете создать свой собственный класс, который наследует nn.Module. Метод forawrd должен точно определять, что происходит при прохождении через слой с учетом входных данных. Это позволяет более точно определять поведение, чем просто накладывать слои один за другим, если это необходимо.

In [ ]:
# create a simple sequential network (`nn.Module` object) from layers (other `nn.Module` objects).
# Here a MLP with 2 layers and sigmoid activation.
net = torch.nn.Sequential(
    torch.nn.Linear(32,128),
    torch.nn.Sigmoid(),
    torch.nn.Linear(128,10))

In [ ]:
# create a more customizable network module (equivalent here)
class MyNetwork(torch.nn.Module):
    # you can use the layer sizes as initialization arguments if you want to
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size,hidden_size)
        self.layer2 = torch.nn.Sigmoid()
        self.layer3 = torch.nn.Linear(hidden_size,output_size)

    def forward(self, input_val):
        h = input_val
        h = self.layer1(h)
        h = self.layer2(h)
        h = self.layer3(h)
        return h

net = MyNetwork(32,128,10)

Сеть отслеживает параметры, и вы можете получить к ним доступ через метод **parameters()**, который возвращает python генератор.

In [ ]:
for param in net.parameters():
    print(param)

Параметры имеют тип Parameter, который в основном является оберткой для тензора. Как Pytorch получает параметры вашей сети? Это просто все атрибуты типа Parameter в вашей сети. Более того, если атрибут имеет тип nn.Module, его параметры добавляются в параметры вашей сети! Вот почему, когда вы определяете сеть путем добавления базовых компонентов, таких как nn.Linear, вам никогда не придется явно определять параметры.

Однако, если вы находитесь в случае, когда ни один из стандартных модулей в Pytorch не делает то, что вам нужно, вы можете определить параметры явно (такое бывает редко). Для записи давайте создадим предыдущий MLP с персонализированными параметрами.

In [ ]:
class MyNetworkWithParams(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(MyNetworkWithParams,self).__init__()
        self.layer1_weights = nn.Parameter(torch.randn(input_size,hidden_size))
        self.layer1_bias = nn.Parameter(torch.randn(hidden_size))
        self.layer2_weights = nn.Parameter(torch.randn(hidden_size,output_size))
        self.layer2_bias = nn.Parameter(torch.randn(output_size))
        
    def forward(self,x):
        h1 = torch.matmul(x,self.layer1_weights) + self.layer1_bias
        h1_act = torch.max(h1, torch.zeros(h1.size())) # ReLU
        output = torch.matmul(h1_act,self.layer2_weights) + self.layer2_bias
        return output

net = MyNetworkWithParams(32,128,10)

Параметры полезны тем, что они должны быть всеми весами сети, которые будут оптимизированы во время обучения. Если вам нужно было использовать тензор в вашем вычислительном графе, который вы хотите оставить постоянным, просто определите его как обычный тензор.

## Training

In [ ]:
net = MyNetwork(32,128,10)

nn.Module также предоставляет функции потерь, например кросс-энтропия.

In [ ]:
x = torch.tensor([np.arange(32), np.zeros(32),np.ones(32)]).float()
y = torch.tensor([0,3,9])
criterion = nn.CrossEntropyLoss()

output = net(x)
loss = criterion(output,y)
print(loss)

nn.CrossEntropyLoss выполняет как softmax, так и саму кросс-энтропию: имея $output$ размера $(n,d)$ и $y$ размера $n$ и значения $0,1,...,d-1$, он вычисляет $\sum_{i=0}^{n-1}log(s[i,y[i]])$ где $s[i,j] = \frac{e^{output[i,j]}}{\sum_{j'=0}^{d-1}e^{output[i,j']}}$

Вы также можете объединить nn.LogSoftmax и nn.NLLLoss, чтобы получить тот же результат. Обратите внимание, что все они используют log-softmax, а не softmax, для стабильности вычислений.

In [ ]:
# equivalent
criterion2 = nn.NLLLoss()
sf = nn.LogSoftmax()
output = net(x)
loss = criterion(sf(output),y)
loss

Теперь, чтобы выполнить backpropagation, просто выполните **loss.backward()**! Это обновит градиенты во всех дифференцируемых тензорах в графе, который, в частности, включает в себя все параметры сети.

In [ ]:
loss.backward()

# Check that the parameters now have gradients
for param in net.parameters():
    print(param.grad)

In [ ]:
# if I forward prop and backward prop again, gradients accumulate :
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

# you can remove this behavior by reinitializing the gradients in your network parameters :
net.zero_grad()
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

Мы делали backpropagation, но все еще не выполняли градиентный спуск. Давайте определим оптимизатор на параметрах сети.

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

print("Parameters before gradient descent :")
for param in net.parameters():
    print(param)

optimizer.step()

print("Parameters after gradient descent :")
for param in net.parameters():
    print(param)

In [ ]:
# In a training loop, we should perform many GD iterations.
n_iter = 1000
for i in range(n_iter):
    optimizer.zero_grad() # equivalent to net.zero_grad()
    output = net(x)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()
    print(loss)

In [ ]:
output = net(x)
print(output)
print(y)

Теперь вы знаете, как тренировать сеть!

## Saving and Loading

In [ ]:
# get dictionary of keys to weights using `state_dict`
net = torch.nn.Sequential(
    torch.nn.Linear(28*28,256),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256,10))
print(net.state_dict().keys())

In [ ]:
# save a dictionary
torch.save(net.state_dict(),'test.pth')
# load a dictionary
net.load_state_dict(torch.load('test.pth'))

## Common issues to look out for

### Type mismatch

In [ ]:
net = nn.Linear(4,2)
x = torch.tensor([1,2,3,4])
y = net(x)
print(y)

In [ ]:
# правильно
net = nn.Linear(4,2)
x = torch.tensor([1.,2.,3.,4.])
# x = torch.tensor([1,2,3,4]).float()
y = net(x)
print(y)

### Не путаем матричное и поэлементное умножение

In [ ]:
x = 2* torch.ones(2,2)
y = 3* torch.ones(2,2)
print(x * y)
print(x.matmul(y))

### Shape mismatch

In [ ]:
x = torch.ones(4,5)
y = torch.arange(5)
print(x.size(), y.size())
print(x+y)
y = torch.arange(4).view(-1,1)
print(x.size(), y.size())
print(x+y)
y = torch.arange(4)
print(x.size(), y.size())
print(x+y) # exception

### View и Transpose ведут себя по-разному

In [ ]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)
print(x.t())
print(x.view(3,2))

### Device mismatch

In [ ]:
net = nn.Sequential(nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,120))
x = torch.ones(256,2048)
y = torch.zeros(256).long()

# правильно добавить:
# x = x.cuda()
# y = y.cuda()

net.cuda()
x.cuda()
crit=nn.CrossEntropyLoss()
out = net(x)
loss = crit(out,y)
loss.backward()

In [ ]:
# неправильный вариант, параметры скрытых слоев не учитываются и не будут обучаться
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
    
            
    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out

net = MyNet(2)
for name, param in net.named_parameters():
    print(name)

In [ ]:
# правильный вариант
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
        self.hidden = nn.ModuleList(self.hidden)
            
    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out
    
net = MyNet(2)
for name, param in net.named_parameters():
    print(name)